In [1]:
import pandas as pd
import numpy as np
import folium, json
import warnings
warnings.filterwarnings('ignore')

In [2]:
# 데이터 전처리

# 각 년도별 엑셀 파일을 하나의 데이터 프레임으로 만들기
df=pd.read_excel('시군구별 월별/시군구별 전력사용량(2004년).xlsx')
df=df.rename(columns=df.iloc[1]).drop(index=[0,1]).reset_index(drop=True)
for i in range(5,23):
    data=pd.read_excel('시군구별 월별/시군구별 전력사용량(20'+'%02d'%(i)+'년).xlsx')
    data=data.rename(columns=data.iloc[1]).drop(index=[0,1]).reset_index(drop=True)
    df=pd.concat([df,data],axis=0)

# 합계만 추출해 각 시군구 월별 전력 사용량 데이터 추출
df=df[df['계약종별']=='합 계'].drop(columns='계약종별').reset_index(drop=True)

# 잘못된 데이터 제거
# 1. 잘못된 지역 제거
df=df[df['시도']!='황해북도']

# 2. 음수인 데이터와 0인 데이터 보간 (양 끝 값은 다음 달이나 이전 달 값으로)
df=df.applymap(lambda x:np.nan if str(x)<="0" else x)
df.iloc[:,3:]=df.iloc[:,3:].interpolate(axis=1).fillna(method='ffill',axis=1).fillna(method='bfill',axis=1)
df

,연도,시도,시군구,1월,2월,3월,4월,5월,6월,7월,8월,9월,10월,11월,12월
0,2014,서울특별시,종로구,158551432.0,150088938.0,134275613.0,128156045.0,122677839.0,140721512.0,160182138.0,163217795.0,145792878.0,126092635.0,125677988.0,151832391.0
1,2014,서울특별시,중구,225229890.0,209930832.0,196322771.0,188568166.0,187575332.0,214581988.0,245564252.0,242897085.0,218258751.0,189737342.0,181960521.0,219028254.0
2,2014,서울특별시,용산구,123467815.0,117993607.0,103794567.0,101444427.0,96682643.0,110524778.0,123836274.0,130716177.0,117370849.0,100717379.0,99874461.0,116880454.0
3,2014,서울특별시,성동구,169857056.0,162151913.0,149672922.0,147075872.0,138919961.0,148721549.0,164188882.0,170612006.0,156793673.0,143670561.0,144715438.0,163871481.0
4,2014,서울특별시,동대문구,132884634.0,128267775.0,112160065.0,109382372.0,102143022.0,112881495.0,124567161.0,134388982.0,121079147.0,105282649.0,106861718.0,123178908.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2069,2022,경상남도,거창군,47136975.0,46763577.0,41135919.0,38684290.0,34859825.0,36597703.0,40323137.0,42721129.0,39643386.0,35555768.0,39599372.0,42243694.0
2070,2022,경상남도,합천군,45636602.0,44733986.0,38465802.0,34305782.0,29169802.0,32617619.0,34618202.0,36619276.0,33292960.0,30032086.0,36205270.0,37485997.0
2071,2022,제주특별자치도,제주시,317382760.0,312863560.0,282631093.0,279015706.0,260799390.0,275897191.0,333992302.0,374992798.0,325234704.0,274007485.0,271833514.0,291704756.0
2072,2022,제주특별자치도,서귀포시,235970974.0,247996771.0,219438140.0,216066207.0,191378889.0,182401818.0,197566647.0,221712922.0,197541741.0,170996896.0,169411175.0,194517951.0


In [3]:
# 해당 연도의 시도별 전력 사용량 지도 시각화 함수 작성

def sidoMap(df,year):
    
    # 해당 연도 데이터 추출
    df_year = df[df['연도']==year]

    # 월별 데이터 합쳐서 연도별 데이터 만들기
    df_year['전력 사용량']=df_year.iloc[:,3:].sum(axis=1)

    # 필요한 칼럼 추출 및 인덱스 재설정
    sido=df_year[['시도','전력 사용량']].reset_index(drop=True)

    # 그룹화 통해 시도별 전력 사용량 합계 구하기
    sido=sido.groupby('시도').sum().reset_index()

    map= folium.Map(location=[35.907757,127.766922],zoom_start=7.3)
    jsonfile = open('korea_geojson2.geojson','r',encoding='utf8')
    jsondata = json.load(jsonfile)

    folium.Choropleth(geo_data=jsondata,
                      data=sido, 
                      columns=['시도','전력 사용량'],
                      key_on='feature.properties.CTP_KOR_NM', # 시도 데이터 들어있는 위치
                      legend_name='전력 사용량',fill_color='YlOrBr').add_to(map)
    return map

In [6]:
# 해당 연도의 시군구별 전력 사용량 지도 시각화 함수 작성

def sigunguMap(df,year):
    
    # 해당 연도 데이터 추출
    df_year = df[df['연도']==year]

    # 월별 데이터 합쳐서 연도별 데이터 만들기
    df_year['전력 사용량']=df_year.iloc[:,3:].sum(axis=1)
    
    # 필요한 칼럼 추출 및 인덱스 재설정
    loc_year=df_year[['시도','시군구','전력 사용량']].reset_index(drop=True)

    # 시도와 시군구를 합쳐 geojson 파일과 매칭할 값 만들기
    loc_year['위치']=loc_year['시도']+loc_year['시군구']
    loc_year=loc_year[['위치','전력 사용량']]

    map= folium.Map(location=[35.907757,127.766922],zoom_start=7.3)
    jsonfile = open('HangJeongDong_ver20230101.geojson','r',encoding='utf8')
    jsondata = json.load(jsonfile)

    # jsondata에서 시도와 시군구 데이터를 가져와 합친 값을 id에 넣기 
    # '경상북도 포항시남구'와 같이 시와 구가 붙어있는 데이터들이 있어 분리 절차 필요
    # '시흥시'가 있으므로 '시'로 구분시 문제 발생 -> 시흥시 따로 처리
    for idx in jsondata['features']:
    #   시흥시가 아닌 지역 처리
        if idx['properties']['sggnm'].find('시흥시') < 0:
            location = idx['properties']['sidonm']+idx['properties']['sggnm'].split('시')[0]
    #       '시'의 앞 부분 가져왔으므로 다시 '시' 붙여주기
            if location[-1]!='구' and location[-1]!='군': 
                location=location+'시'
    #   시흥시 처리
        else: location = '경기도시흥시'
        idx['id']=location

    folium.Choropleth(geo_data=jsondata,
                      data=loc_year, 
                      columns=['위치','전력 사용량'],
                      key_on='feature.id',
                      legend_name='전력 사용량',fill_color='YlOrBr').add_to(map)
    return map